In [1]:
import pandas as pd
from datetime import datetime, timedelta
%matplotlib inline
import json
import matplotlib.pyplot as plt
import pickle

In [27]:
complete_df = pd.read_csv("data/116_edit",sep = "\t" , header = 0)
complete_df['calibrated_time'] = pd.to_datetime(complete_df['calibrated_time'])

In [15]:
complete_df[complete_df.licence_plate_number == complete_df.licence_plate_number.unique().tolist()[0]].head()

,raw_index,route,licence_plate_number,calibrated_time,longitude,latitude,delta_lon,delta_lat,delta_time
0,19,116,粤E19204,2015-07-01 00:01:35,113.14631,23.02181,0.0,0.0,5
1,39,116,粤E19204,2015-07-01 00:01:40,113.14631,23.02181,0.0,0.0,5
2,57,116,粤E19204,2015-07-01 00:01:45,113.14631,23.02181,0.0,0.0,5
3,82,116,粤E19204,2015-07-01 00:01:50,113.14631,23.02181,0.0,0.0,5
4,102,116,粤E19204,2015-07-01 00:01:55,113.14631,23.02181,0.0,0.0,5


In [17]:
start_range = [(113.145, 113.147), (23.0205, 23.0225)]
end_range = [(113.0165, 113.0185), (23.046, 23.048)]

In [38]:
shifts = []
for bus in complete_df.licence_plate_number.unique().tolist()[:]:
    temp_bus = complete_df[complete_df.licence_plate_number == bus]
    for i in temp_bus.index:
        if temp_bus.loc[i,"longitude"] < start_range[0][1] and temp_bus.loc[i,"longitude"] > start_range[0][0] and temp_bus.loc[i,"latitude"] < start_range[1][1] and temp_bus.loc[i,"latitude"] > start_range[1][0]:
            # if within departure station, renew the shift list
#             temp = [(i, temp_bus.loc[i,"longitude"], temp_bus.loc[i,"latitude"])]
            temp = [i]
        elif temp_bus.loc[i,"longitude"] < end_range[0][1] and temp_bus.loc[i,"longitude"] > end_range[0][0] and temp_bus.loc[i,"latitude"] < end_range[1][1] and temp_bus.loc[i,"latitude"] > end_range[1][0]:
            # if within terminal station, check if the length of the list satisfied our requirement
            if len(temp)> 100 :
                if (temp_bus.loc[temp[-1],"calibrated_time"] - temp_bus.loc[temp[0],"calibrated_time"]).total_seconds()>3600 and (temp_bus.loc[temp[-1],"calibrated_time"] - temp_bus.loc[temp[0],"calibrated_time"]).total_seconds()<9600 :
                    shifts.append(temp)
            temp = []

        elif len(temp)>0:
            # if between the two stations, add it to the list
#             temp.append((i, temp_bus.loc[i,"longitude"], temp_bus.loc[i,"latitude"]))
            temp.append(i)

#     print complete_df[complete_df.licence_plate_number == bus].shape


In [19]:
# complete_df[complete_df.licence_plate_number == bus]

In [39]:
len(shifts)

402

In [40]:
valid_shift = []

# further filter previously selected shifts
for shift in shifts[:]:
    # find out the index of each data point
    ind = [s for s in shift]
    # standardize the time based on the starting point
    timelist = list(pd.to_datetime(complete_df.loc[ind]["calibrated_time"]))
    timelist = [timedelta.total_seconds(t-timelist[0]) for t in timelist if timedelta.total_seconds(t-timelist[0])< 8000 and timedelta.total_seconds(t-timelist[0])> 0]
    # extract lat and lon
    xlist = list(complete_df.loc[ind]["longitude"])
    ylist = list(complete_df.loc[ind]["latitude"])
    temp = zip(xlist, ylist, timelist)
    
    if max(xlist)>113.2:
        continue
    valid_shift.append(temp)

# number of valid shifts
print len(valid_shift)

402


In [43]:
datalist = [[{"x":point[0], "y":point[1], "t":point[2]} for point in shift] for shift in valid_shift]
fout =  open("data/116_full.json","w")
json.dump(datalist, fout)

In [44]:
f = open("data/116_full.json","r")
d = json.loads(f.read())
print d

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
